In [1]:
import requests

### Specify the server

In [2]:
server = 'http://cohd.nsides.io/api/v1'  # COHD server

### Functions to perform REST API requests

In [3]:
def find_concept(concept_name):
    url = f'{server}/omop/findConceptIDs'
    query_params = {'q': concept_name}
    r = requests.get(url, query_params)
    return r.json()

def get_concept(concept_ids):
    url = f'{server}/omop/concepts'
    concept_ids_list = list(concept_ids)
    concept_ids_string = ','.join([str(x) for x in concept_ids_list])
    query_params = {'q': concept_ids_string}
    r = requests.get(url, query_params)
    return r.json()

def get_concept_frequency(concept_ids):
    url = f'{server}/frequencies/singleConceptFreq'
    concept_ids_list = list(concept_ids)
    concept_ids_string = ','.join([str(x) for x in concept_ids_list])
    query_params = {'q': concept_ids_string}
    r = requests.get(url, query_params)
    return r.json()

def get_paired_concepts_frequency(concept_id_1, concept_id_2):
    url = f'{server}/frequencies/pairedConceptFreq'
    query_params = {'q': f'{concept_id_1!s},{concept_id_2!s}'}
    r = requests.get(url, query_params)
    return r.json()

def get_associated_concepts_freq(concept_id):
    url = f'{server}/frequencies/associatedConceptFreq'
    query_params = {'q': f'{concept_id!s}'}
    r = requests.get(url, query_params)
    return r.json()

def get_associated_concept_domain_freq(concept_id, domain_id):
    url = f'{server}/frequencies/associatedConceptDomainFreq'
    query_params = {'concept_id': str(concept_id), 'domain': domain_id}
    r = requests.get(url, query_params)
    return r.json()

### Find concepts similar to cancer

In [4]:
r = find_concept('cancer')
for x in r['results']:
    print(f'{x["concept_id"]!s}: {x["concept_name"]}')

192855: Cancer in situ of urinary bladder
2008271: Injection or infusion of cancer chemotherapeutic substance


### Look up a few concepts

In [5]:
concept_ids = [313217, 192855, 2008271]
r = get_concept(concept_ids)
for x in r['results']:
    print(f'{x["concept_id"]!s} - {x["domain_id"]}: {x["concept_name"]}')

192855 - Condition: Cancer in situ of urinary bladder
313217 - Condition: Atrial fibrillation
2008271 - Procedure: Injection or infusion of cancer chemotherapeutic substance


### Get the frequency of atrial fibrillation

In [6]:
concept_ids = [313217] # Atrial fibrillation
r = get_concept_frequency(concept_ids)
for x in r['results']:
    concept_count = x['concept_count']
    concept_frequency = x['concept_frequency']
    
    print(f'concept_count: {concept_count!s}')
    print(f'concept_frequency: {concept_frequency!s}')

concept_count: 58156
concept_frequency: 0.010912897789442022


### Get the frequency of a pair of concepts

In [7]:
concept_id_1 = 192855  # Cancer in situ of urinary bladder
concept_id_2 = 2008271 # Injection or infusion of cancer chemotherapeutic substance
r = get_paired_concepts_frequency(concept_id_1, concept_id_2)
for x in r['results']:
    concept_count = x['concept_count']
    concept_frequency = x['concept_frequency']
    
    print(f'concept_count: {concept_count!s}')
    print(f'concept_frequency: {concept_frequency!s}')

concept_count: 27
concept_frequency: 5.066514896398214e-06


### Get all frequency information for pairs of concepts including a given concept

In [8]:
concept_id = 192855  # Cancer in situ of urinary bladder
r = get_associated_concepts_freq(concept_id)
print('concept_id, associated_concept_id, associated_concept_name, associated_concept_domain, count, frequency')
for x in r['results']:
    associated_concept_id = x['associated_concept_id']
    associated_concept_name = x['associated_concept_name']
    associated_concept_domain = x['associated_concept_domain']
    concept_count = x['concept_count']
    concept_frequency = x['concept_frequency']
    
    print(f'{concept_id!s}, {associated_concept_id!s}, {associated_concept_name}, {associated_concept_domain}, {concept_count!s}, {concept_frequency!s}')

concept_id, associated_concept_id, associated_concept_name, associated_concept_domain, count, frequency
192855, 197508, Malignant tumor of urinary bladder, Condition, 690, 0.00012947760290795438
192855, 38003564, Not Hispanic or Latino, Ethnicity, 625, 0.00011728043741662534
192855, 8507, MALE, Gender, 611, 0.00011465335561849293
192855, 8527, White, Race, 586, 0.00010996213812182791
192855, 81902, Urinary tract infectious disease, Condition, 502, 9.419964733303347e-05
192855, 320128, Essential hypertension, Condition, 466, 8.744429413783586e-05
192855, 79864, Hematuria syndrome, Condition, 386, 7.24323981485078e-05
192855, 19041324, Acetaminophen 325 MG Oral Tablet [Tylenol], Drug, 380, 7.13065059493082e-05
192855, 40231925, Acetaminophen 325 MG / Oxycodone Hydrochloride 5 MG Oral Tablet, Drug, 356, 6.68029371525098e-05
192855, 40240688, Docusate Sodium 100 MG Oral Capsule, Drug, 350, 6.567704495331019e-05
192855, 19070869, Ondansetron 2 MG/ML Injectable Solution, Drug, 327, 6.1361124

### For a given concept, get paired frequency information of all associated concepts from a given domain

In [9]:
concept_id = 192855  # Cancer in situ of urinary bladder
domain_id = 'Procedure'
r = get_associated_concept_domain_freq(concept_id, domain_id)
print('concept_id, associated_concept_id, associated_concept_name, count, frequency')
for x in r['results']:
    associated_concept_id = x['associated_concept_id']
    associated_concept_name = x['associated_concept_name']
    concept_count = x['concept_count']
    concept_frequency = x['concept_frequency']
    
    print(f'{concept_id!s}, {associated_concept_id!s}, {associated_concept_name}, {concept_count!s}, {concept_frequency!s}')

concept_id, associated_concept_id, associated_concept_name, count, frequency
192855, 2003789, Other transurethral excision or destruction of lesion or tissue of bladder, 120, 2.2517843983992063e-05
192855, 2003783, Other cystoscopy, 87, 1.6325436888394247e-05
192855, 2109773, Cystourethroscopy, with fulguration (including cryosurgery or laser surgery) or treatment of MINOR (less than 0.5 cm) lesion(s) with or without biopsy, 87, 1.6325436888394247e-05
192855, 2109766, Cystourethroscopy (separate procedure), 86, 1.6137788188527647e-05
192855, 2003784, Closed [transurethral] biopsy of bladder, 83, 1.5574842088927845e-05
192855, 2109771, Cystourethroscopy, with biopsy(s), 71, 1.3323057690528638e-05
192855, 2109194, Colonoscopy, flexible; diagnostic, including collection of specimen(s) by brushing or washing, when performed (separate procedure), 52, 9.757732393063227e-06
192855, 2109774, Cystourethroscopy, with fulguration (including cryosurgery or laser surgery) and/or resection of; SMALL